In [7]:
# 필요한 패키지 Import
import tensorflow as tf  # 본 실습에서 사용한 tensorflow는 1.13.1 (또는 1.14.1) 버전임 

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata  # Unicode Character Database를 접근할 수 있도록 하는 모듈
import re          # regular expression package
import numpy as np
import os
import time
import string

import plotly  # plotly는 3.10.0 버전을 사용함, Plotly Python Open Source Graphing Library
import plotly.plotly as py

# iplot: plotly 렌더링 모듈의 구버전
from plotly.offline import init_notebook_mode, iplot 

#  init_notebook_mode: Jupyter Notebook 환경에서 Plotly를 사용하기 위해 init_notebook_mode를 호출
plotly.offline.init_notebook_mode(connected=True)

# graph_objs: scatter plot을 그리기 위해 사용하는 plotly의 모듈 중 하나

import plotly.graph_objs as go

In [2]:
tf.__version__

'2.4.0'

In [8]:
plotly.__version__

'3.10.0'

In [16]:
file_path = './datasets/14_영한번역_소스코드/kor-eng/kor.txt' # 데이터셋의 위치를 명시함

In [17]:
# txt 파일에서 데이터를 읽고 정제함
# 데이터셋을 읽고(read), 문자열의 양 끝에 존재하는 공백과 \n 제거(strip) 및 \n로 분리(split)하여 라인 생성
lines = open(file_path, encoding='UTF-8').read().strip().split('\n') 
# 각 라인을 \t로 분리했을 때 처음 2개 string만 추출 
lines = [l.split('\t')[:2] for l in lines]
# 영문장과 해당 번역문을 \t로 묶어 하나의 라인 형성
lines = [l[0] + '\t' + l[1] for l in lines]
# 200번째 라인부터 10개 라인 추출하여 출력
lines[200:210] 

['Some water, please.\t물 좀 주세요, 제발.',
 'The light went out.\t전등이 꺼졌다.',
 'The night was cold.\t그날 밤은 추웠어.',
 'Tie your shoelaces.\t신발끈을 묶으세요.',
 'We were retreating.\t우리는 후퇴하고 있었다.',
 "We've been worried.\t계속 걱정했어.",
 'When will you come?\t언제쯤 올거야?',
 'Whose book is this?\t이것은 누구의 책입니까?',
 'Can I have this cup?\t이 컵 가져도 돼요?',
 'Do you like English?\t영어 좋아해요?']

In [18]:
len(lines)

904

In [19]:
# 특수 문자와 숫자를 제거하기 위한 작업
exclude = set(string.punctuation) # 모든 특수 문자를 exclude 변수에 저장
remove_digits = str.maketrans('', '', string.digits) # 숫자로 구성된 Set을 remove_digits에 저장

In [20]:
# 영어 문장을 전처리
def preprocess_eng_sentence(sent):
    '''영어 문장 전처리 함수'''
    sent = sent.lower() # 영어 문자를 소문자로 변환
    sent = re.sub("'", '', sent) # "'"부호가 있으면 삭제함
    sent = ''.join(ch for ch in sent if ch not in exclude) # 특수부호를 삭제
    sent = sent.translate(remove_digits) # 모든 숫자를 제거함
    sent = sent.strip() # 문자열의 양 끝에 존재하는 공백과 \n 제거
    sent = re.sub(" +", " ", sent) # 특정 패턴의 문자열(" +")을 다른 문자열(" ")로 바꾸기 
    sent = '<start> ' + sent + ' <end>' # <start> 와 <end> token을 추가함
    return sent

In [21]:
# 한글 문장을 전처리
def preprocess_kor_sentence(sent):
    '''Function to preprocess Korean sentence'''
    sent = re.sub("'", '', sent) # "'"부호가 있으면 삭제함
    sent = ''.join(ch for ch in sent if ch not in exclude) # 특수부호를 삭제
    sent = sent.strip() # 문자열의 양 끝에 존재하는 공백과 \n 제거
    sent = re.sub(" +", " ", sent) # 특정 패턴의 문자열(" +")을 다른 문자열(" ")로 바꾸기
    sent = '<start> ' + sent + ' <end>' # <start> 와 <end> token을 추가함
    return sent

In [22]:
# 정제된 영어와 한국어 문장을 쌍으로 생성
sent_pairs = []
for line in lines:
    sent_pair = []
    eng, mar = line.split('\t') # 영문장 및 번역문 쌍을 각각 eng와 mar로 나누어 저장 
    
    # 영어 문장
    eng = preprocess_eng_sentence(eng) # 영문장 eng 전처리
    sent_pair.append(eng) # 전처리 된 영문장을 sent_pair 리스트의 첫번째 원소로 저장
    # 한글 문장
    mar = preprocess_kor_sentence(mar) # 번역문 mar 전처리
    sent_pair.append(mar) # 전처리 된 번역문을 sent_pair 리스트의 두번째 원소로 저장
    sent_pairs.append(sent_pair) # 전처리가 끝난 영문장과 번역문으로 이루어진 리스트를 sent_pairs 리스트에 저장

# 200번째 라인부터 10개 라인 추출
sent_pairs[200:210]

[['<start> some water please <end>', '<start> 물 좀 주세요 제발 <end>'],
 ['<start> the light went out <end>', '<start> 전등이 꺼졌다 <end>'],
 ['<start> the night was cold <end>', '<start> 그날 밤은 추웠어 <end>'],
 ['<start> tie your shoelaces <end>', '<start> 신발끈을 묶으세요 <end>'],
 ['<start> we were retreating <end>', '<start> 우리는 후퇴하고 있었다 <end>'],
 ['<start> weve been worried <end>', '<start> 계속 걱정했어 <end>'],
 ['<start> when will you come <end>', '<start> 언제쯤 올거야 <end>'],
 ['<start> whose book is this <end>', '<start> 이것은 누구의 책입니까 <end>'],
 ['<start> can i have this cup <end>', '<start> 이 컵 가져도 돼요 <end>'],
 ['<start> do you like english <end>', '<start> 영어 좋아해요 <end>']]

In [23]:
# 단어에서 인텍스의 매핑과 인텍스에서 단어의 매핑을 생성하는 클래스 정의
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        # 변수 및 형태(타입) 선언
        self.lang = lang # lang 
        self.word2idx = {} # 단어-인덱스 : 딕셔너리 타입
        self.idx2word = {} # 인덱스-단어 : 딕셔너리 타입
        self.vocab = set() # 사전 구축(데이터의 중복을 허용하지 않으며 저장되는 데이터에 대한 순서가 없음) 

        self.create_index() # 인덱스 생성 함수 선언
    
    # 단어의 Index를 생성
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' ')) # 띄어쓰기(' ')로 나뉘어 생성된 단어(어절)를 사전에 추가

        self.vocab = sorted(self.vocab) # 알파벳 순서대로 사전(단어) 정렬

        self.word2idx['<pad>'] = 0 # <pad> 단어에 인덱스 0을 지정 (예시: word2idx = {'<pad>': 0})
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # 위에서 생성된 사전(vocab)의 각 단어에 인덱스 생성(1부터)

        for word, index in self.word2idx.items():
            # word2idx의 각 단어에 지정된 인덱스를 idx2word에 인덱스: 단어 형식으로 지정
            # 예시: idx2word = {0:'<pad>'}
            self.idx2word[index] = word 

In [24]:
# 텐서의 최대 길이를 계산하는 함수
def max_length(tensor):
    return max(len(t) for t in tensor)

In [25]:
# 입력 데이터와 타겟 데이터에 대해 Indexing 작업을 진행
# 패딩(Padding) 작업 진행
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, ma in pairs)
    targ_lang = LanguageIndex(ma for en, ma in pairs)
    
    # Vectorize the input and target languages
    
    # 영어 문장
    # "<start> no way <end>" => [2, 656, 1079, 1]
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]
    
    # 한국어 문장
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]
    
    # 입력 텐서(영어 문장), 출력 텐서(한국어 문장)에서 최대 길이를 각각 계산함 
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # 텐서의 최대 길이에 맞추어 패딩(Padding)을 추가하는 작업
    # 해당 단어에 대해 Index 번호를 추가, 없는 부분에 0 값으로 패딩 
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, # 영어문장
                                                                 maxlen=max_length_inp, # 영어문장 텐서의 최대 길이 
                                                                 padding='post') # 'pre' 또는 'post': pre를 쓰면 시퀀스 앞에 패딩을 하고 post를 쓰면 시퀀스 뒤에 패딩
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, # 번역문
                                                                  maxlen=max_length_tar, # 번역문 텐서의 최대 길이 
                                                                  padding='post') # 'pre' 또는 'post': pre를 쓰면 시퀀스 앞에 패딩을 하고 post를 쓰면 시퀀스 뒤에 패딩
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [26]:
# load_dataset() 함수 호출
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

print(input_tensor)
print("The max length of Input Tensor: ", max_length_inp)
print("The max length of Output Tensor: ", max_length_targ)

[[   2 1099    1 ...    0    0    0]
 [   2  436    1 ...    0    0    0]
 [   2  656 1079 ...    0    0    0]
 ...
 [   2 1002  665 ...    0    0    0]
 [   2  468  981 ...  997  822    1]
 [   2  497  660 ... 1129    1    0]]
The max length of Input Tensor:  19
The max length of Output Tensor:  17


In [27]:
# 훈련용 데이터와 테스트 데이터를 9대1로 분할
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, \
                                                                                                target_tensor, test_size=0.1
                                                                                             , random_state = 101)

# 분할된 데이터 개수를 보여줌
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(813, 813, 91, 91)

In [28]:
# 하이퍼 파라미터 설정
BUFFER_SIZE = len(input_tensor_train) # 학습데이터 사이즈
BATCH_SIZE = 64 # batch 사이즈
N_BATCH = BUFFER_SIZE//BATCH_SIZE # batch 수
'''
N_BATCH는 각각의 batch 그룹에 들어가는 배치의 수
BUFFER_SIZE = 813, BATCH_SIZE = 64로 설정하였으므로 813//64 = 12
N_BATCH = 12, 즉, 하나의 배치그룹은 12개의 배치를 가짐
'''
embedding_dim = 256 # 임베딩 차원
units = 1024 # 신경망 유닛 수

# 입력 데이터(영어)의 전체 단어의 개수
vocab_inp_size = len(inp_lang.word2idx)
# 출력 데이터(한글)의 전체 단어의 개수
vocab_tar_size = len(targ_lang.word2idx)

# tf.data.Dataset을 생성하는 함수로, numpy 배열이 있고 그걸 tensorflow로 넣는 기본 케이스
# 데이터를 특성(feature)과 라벨(label)로 나누어 사용하는 경우처럼, 한 개 이상의 numpy 배열을 넣을 수 있음
# 해당 방법을 사용하면 학습 속도가 빨라지는 장점이 있음
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
print(dataset)

# 배치 데이터를 생성
# 연속의 원소를 batch에 batch_size만큼 합치는 작업
# 마지막 batch가 batch_size보다 작은 경우 drop
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
print(dataset)

<ShuffleDataset shapes: ((19,), (17,)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((64, 19), (64, 17)), types: (tf.int32, tf.int32)>


In [29]:
def gru(units):
    # 정의한  GRU 구조.
    return tf.keras.layers.GRU(units, # 결과값 차원 수(dimensionality of the output space)
                               return_sequences=True, # 마지막 시퀀스를 출력할 것인지, 아니면 전체 시퀀스를 출력할 것인지 여부
                               return_state=True, # output 외에도 최후 state를 출력할 것인지 여부
                               recurrent_activation='sigmoid', # recurrent 단계에서 activation function
                               recurrent_initializer='glorot_uniform') # recurrent_kernel 가중치 행렬 초기화

In [30]:
# 인코더 클래스를 정의
class Encoder(tf.keras.Model):
    
    # 전체 영어 단어의 길이, 임베딩 차원, 유닛 개수, 배치 사이즈를 입력으로 함
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz # 배치 사이즈
        self.enc_units = enc_units # 유닛(unit) 개수
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # 임베딩 층, (영어 단어의 개수, 임베딩 차원)
        self.gru = gru(self.enc_units) # GRU 층
    
    # 인코더 호출 함수
    # 해당 함수는 임베딩 층과 GRU 층으로 구성됨
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    # Hidden state를 초기화
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [31]:
# 디코더 클래스를 정의
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz # 배치 사이즈
        self.dec_units = dec_units # 유닛(unit) 개수
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # 임베딩 층
        self.gru = gru(self.dec_units) # GRU 층
        self.fc = tf.keras.layers.Dense(vocab_size) # 전체 한국어 단어의 개수
        
        # Attention에서 사용함
        self.W1 = tf.keras.layers.Dense(self.dec_units) # encoder ouptput 가중치
        self.W2 = tf.keras.layers.Dense(self.dec_units) # RNN 셀 각각의 output을 입력값으로 취하는 Feed-Forward Fully Connected Layer
        self.V = tf.keras.layers.Dense(1) # context vector
        
    def call(self, x, hidden, enc_output):
        '''
        해당 레이어의 output을 각 RNN 셀의 Score로 결정.
        출력된 score에 Softmax를 취하여 0~1값으로 변환하고 각각의 attention weight로 변환.
        attention weight와 hidden state를 곱해서 Context vector 획득 후, 디코더에 input으로 들어감.
        결과가 정답이 아니면 attention weight를 backpropagate방식으로 다시 조절하면서 학습하며, 
        따라서 디코더의 hidden state가 attention weight 계산에 영향을 줌.
        '''
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # 스코어를 구하기 위해 차원(1) 추가
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # tanh(FC(EO) + FC(H))을 self.V에 적용하여 마지막 축에 1을 가짐
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        # 소프트맥스 스코어 산출
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        # 가중치 적용
        context_vector = attention_weights * enc_output
        # 텐서의 차원들을 탐색하며 개체들의 총합을 계산함
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # 임베딩 후 x의 shape == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # 임베딩 된 x와 context_vector를 모두 붙이고 난 뒤 x의 shape == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # LSTM의 변형인 GRU(Gated Recurrent Unit)로 x를 보냄
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        # hidden state를 0으로 채워 초기화
        return tf.zeros((self.batch_sz, self.dec_units))

In [32]:
# 인코더와 디코더 객체를 생성
# 'vocab_inp_size' => 전체 영어 단어의 개수, 'vocab_tar_size' => 전체 한국어 단어의 개수
# 'embedding_dim' => 256, 'units' => 1024, 'BATCH_SIZE' => 64
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE) 

In [35]:
# 최적화 함수
optimizer = tf.optimizers.Adam()

# 손실함수 정의
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0) # 원소 단위로 비교 연산을 만족하면 True 반환, 만족하지 않으면 False 반환함
    # softmax 산출 후 cross_entropy를 구하는 함수
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [36]:
# 체크 포인트 생성
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [37]:
EPOCHS = 20

# Epoch 만큼 반복해 모델을 훈련함
for epoch in range(EPOCHS):
    start = time.time()
    
    # 인코더의 히든 상태를 초기화
    hidden = encoder.initialize_hidden_state()
    # 전체 손실을 저장하는 변수를 정의하고 0을 초기값으로 설정
    total_loss = 0
    
    # 반복문을 사용해 배치 사이즈만큼의 영어 문장과 한국어 문장을 가져옴
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        # GradientTape는 자동 미분을 계산하는 API
        # 수동으로 그래디언트를 게산해 가중치를 업데이트하기 위함
        with tf.GradientTape() as tape:
            
            # 배치 사이즈 만큼인 영어 문장과 hidden 상태값을 사용해 인코더를 호출
            # 인코더 출력과 인코더 히든을 반환
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            # 디코더의 입력을 준비 (디코더 모델의 첫번째 스텝의 입력을 '<start>'의 index 번호로 설정함)
            # shape은 (64, 1)
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)    
            
            # Teacher forcing - feeding the target as the next input
            # 디코더를 step by step으로 호출해 예측 값에 대한 손실을 계산함
            for t in range(1, targ.shape[1]):
                
                # 인코딩 출력을 디코더에 입력함
                # 디코더의 입력, 히든 값, 인코더의 출력을 사용해 디코더를 호출
                # 그 다음 예측 값과 히든 값을 반환
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                # 타겟 값과 예측 값을 비교해 손실 값을 계산
                # 게산된 손실 값을 누적함
                loss += loss_function(targ[:, t], predictions)
                
                # 그 다음 디코더의 입력을 설정
                # 현재 스탭의 타겟으로 설정함 (예: target: [<start>, a,, b, c, d] 일 때 
                # 디코더의 첫번째 스텝의 입력: <start>, 두 번째 입력: a, 세 번째 입력: b, )
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        # 배치 손실을 계산
        batch_loss = (loss / int(targ.shape[1]))
        # 전체 손실의 누적 합
        total_loss += batch_loss
        
        # 업데이트를 하기 위한 인코더와 디코더의 가중치를 가지고 옴
        variables = encoder.variables + decoder.variables
        
        # 손실을 사용해 기울기 값을 계산함
        gradients = tape.gradient(loss, variables)
        
        # 기울기 값을 사용해 가중치를 업데이트함
        optimizer.apply_gradients(zip(gradients, variables))
        
        # 100번째 배치마다 출력
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # 매 Epoch마다 모델을 체크포인트 폴더에 저장
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.3794
Epoch 1 Loss 2.2221
Time taken for 1 epoch 64.12617707252502 sec

Epoch 2 Batch 0 Loss 2.2513
Epoch 2 Loss 1.9918
Time taken for 1 epoch 69.75529956817627 sec

Epoch 3 Batch 0 Loss 1.8079
Epoch 3 Loss 1.8694
Time taken for 1 epoch 63.175485134124756 sec

Epoch 4 Batch 0 Loss 1.7951
Epoch 4 Loss 1.7686
Time taken for 1 epoch 51.86296486854553 sec

Epoch 5 Batch 0 Loss 1.7496
Epoch 5 Loss 1.6851
Time taken for 1 epoch 67.77093362808228 sec

Epoch 6 Batch 0 Loss 1.6983
Epoch 6 Loss 1.6285
Time taken for 1 epoch 65.52886605262756 sec

Epoch 7 Batch 0 Loss 1.7053
Epoch 7 Loss 1.5941
Time taken for 1 epoch 55.870238065719604 sec

Epoch 8 Batch 0 Loss 1.5944
Epoch 8 Loss 1.5417
Time taken for 1 epoch 59.78965210914612 sec

Epoch 9 Batch 0 Loss 1.5488
Epoch 9 Loss 1.5166
Time taken for 1 epoch 63.44730353355408 sec

Epoch 10 Batch 0 Loss 1.5154
Epoch 10 Loss 1.4739
Time taken for 1 epoch 71.14693236351013 sec

Epoch 11 Batch 0 Loss 1.3989
Epoch 11 Loss 1.4327
Time t

In [38]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [39]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    """훈련한 모델을 사용해 번역 작업을 진행하는 함수"""
    
    # heat map을 그리기 위한 matrix 초기화
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    # 입력 inputs의 인덱스를 단어로 디코딩해 문장을 생성함
    # 예: [2, 1099, 1, 0, 0 ,..] => '<start> who <end>'
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    
    # 마지막에 추가된 스페이스를 지움 
    sentence = sentence[:-1]
    
    #NumPy ndarray 자료형을 변환
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''
    
    # 모든 원소의 값이 0인 텐서를 생성
    hidden = [tf.zeros((1, units))]
    
    # inputs를 encoder에 입력
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    # Step-by-step으로 decoder를 실행해 문장을 번역함
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # attention weight heat map를 그리기 위해 매 step의 attention 가중치를 저장함
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        
        # 텐서 안에서 predictions[0]을 따라 가장 큰 값의 인덱스를 찾기
        predicted_id = tf.argmax(predictions[0]).numpy()

        # decoder가 예측한 인덱스를 단어로 디코딩함
        result += targ_lang.idx2word[predicted_id] + ' '

        # docoder 실행 중 '<end>'가 나타나면 문장이 끝이므로 실행 중지
        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # 예측한 출력을 decoder의 다음 step의 입력으로 사용하기 위해 dimension을 expand 함
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [40]:
def predict_random_val_sentence():
    """테스트셋에서 랜덤으로 문장을 선택해 번역하고 attention heatmap을 그리는 함수"""
    
    actual_sent = ''
    
    # 테스트셋에서 랜덤으로 인코딩 된 문장을 선택
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    
    # evaluate 함수를 사용해 번역 작업을 진행함
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp,\
                                                max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))  # 입력 문장의 <start>와 <end>를 제거하고 출력
    print('Predicted translation: {}'.format(result[:-6]))  # 번역한 문장에서 <end>를 제거하고 출력
    # 랜덤으로 선택한 입력 문장의 정답 레이블을 인텍스에서 단어로 디코딩함
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]   # 타깃 문장의 <start>와 <end>를 제거하고 출력
    print('Actual translation: {}'.format(actual_sent))
    
    # attention weight 
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    
    # 영문장 및 번역문 토큰 리스트 지정
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]
    
    # plotly를 사용해 attention weight의 heat map를 생성함
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='Reds')
    data=[trace]
    iplot(data)

In [43]:
predict_random_val_sentence()

Input: how many times do i need to repeat it
Predicted translation: 톰은 메리가 피아노 치는 것을 믿을 수 없었다 
Actual translation: 몇 번을 반복해야 해


In [44]:
predict_random_val_sentence()

Input: tom doesnt know why im here yet
Predicted translation: 톰은 메리가 나의 쉽지는 않아 
Actual translation: 톰은 제가 왜 여기 있는지 아직 모릅니다


In [45]:
predict_random_val_sentence()

Input: i hope that i can do it
Predicted translation: 나는 톰에게 내 마지막 진행되었어요 
Actual translation: 내가 할 수 있길 바라
